In [ ]:
def kbdi_single_grid(tmax_1d, pr_1d):
    # Ensure inputs are NumPy arrays
    T = np.asarray(tmax_1d)
    PR = np.asarray(pr_1d)

    if not np.all(np.isfinite(PR)):
        return np.full_like(PR, np.nan)

    # Create a time index
    time_index = np.arange(len(PR),dtype='float32')

    # 7-day rolling precipitation sum
    ndays = 7
    pr_thresh = 8.0  # inches
    pr_weeksum = np.convolve(PR, np.ones(ndays), mode='valid').astype('float32')
    pr_weeksum = np.concatenate([np.full(ndays - 1, np.nan), pr_weeksum]).astype('float32')

    try:
        day_int = np.where(pr_weeksum > pr_thresh)[0][0].astype('int32')
    except IndexError:
        return np.full_like(PR, np.nan)

    # Rain mask and consecutive rain days
    rainmask = np.where(PR > 0, 1, 0).astype('int32')
    cumsum = np.cumsum(rainmask)
    reset = np.where(rainmask == 0, cumsum, np.nan)
    reset = np.maximum.accumulate(np.where(np.isnan(reset), -1, reset))
    rr = cumsum - reset

    # Categorize rain days
    cat = np.where(rr >= 3, 5, rr)
    consec_day2 = np.where(rr == 2)[0].astype('int32')
    consec_day1 = consec_day2 - 1
    cat[consec_day2] = 5
    cat[consec_day1] = 5
    cat = np.where(cat == 5, 2, cat)

    # Pnet calculation
    acc_thresh = 0.2  # inches
    pnet = np.copy(PR)
    pnet[cat == 1] = pnet[cat == 1] - acc_thresh
    pnet = np.where(pnet < 0, 0, pnet)

    # Adjust for consecutive rain days
    consec_inds = np.where(cat == 2)[0].astype('int32')
    accpr = 0.0
    thresh_flag = False

    for i, ind in enumerate(consec_inds):
        accpr += PR[ind]
        if accpr <= acc_thresh and not thresh_flag:
            pnet[ind] = 0
        elif accpr > acc_thresh and not thresh_flag:
            accpr -= acc_thresh
            pnet[ind] = accpr
            thresh_flag = True
        else:
            pnet[ind] = PR[ind]
        if i != len(consec_inds) - 1 and consec_inds[i + 1] != consec_inds[i] + 1:
            accpr = 0.0
            thresh_flag = False

    # Mean annual precipitation (approximate)
    days_per_year = 365
    n_years = len(PR) // days_per_year
    ann_pr = []
    for i in range(n_years):
        year_data = PR[i * days_per_year:(i + 1) * days_per_year]
        if np.count_nonzero(~np.isnan(year_data)) >= 360:
            ann_pr.append(np.nansum(year_data))
    if len(ann_pr) == 0:
        return np.full_like(PR, np.nan)
    mean_ann_pr = np.mean(ann_pr)

    # KBDI calculation
    KBDI = np.full_like(PR, np.nan)
    if day_int < len(PR):
        KBDI[day_int] = 0
    else:
        return KBDI

    denominator = 1 + 10.88 * np.exp(-0.0441 * mean_ann_pr)
    for it in range(day_int + 1, len(PR)):
        Q = max(0, KBDI[it - 1] - pnet[it] * 100)
        numerator = (800 - Q) * (0.968 * np.exp(0.0486 * T[it]) - 8.3)
        KBDI[it] = Q + (numerator / denominator) * 1e-3
    
    return KBDI